In [1]:
import pandas as pd
import numpy as np
from gurobipy import GRB
import gurobipy as gp

In [2]:
holds = [
    ['Front', 10, 6800],
    ['Center',16,8700],
    ['Back',8,5300],
]
holds = pd.DataFrame(holds, columns=['Hold','Weight','Volume'])

In [3]:
shipment = [
    ['C1',18,480,310],
    ['C2',15,650,380],
    ['C3',23,580,350],
    ['C4',12,390,285],
]
shipment = pd.DataFrame(shipment, columns=['Cargo','Weight','Volume','Profit'])

In [4]:
m = gp.Model("Cargo")

Restricted license - for non-production use only - expires 2023-10-25


In [5]:
## Allocating decision variables.
## x[i][j] -> put x[i][j] proportion of cargo i to hold j
x = m.addMVar(shape = (4, 3), vtype=GRB.CONTINUOUS, name="decision")

In [6]:
## Decision variables should be proportions, so in the range [0,1]
for i in range(len(shipment)):
    for j in range(len(holds)):
        m.addConstr(
            x[i][j] >= 0
        )

In [7]:
for i in range(len(shipment)):
    m.addConstr(
        gp.quicksum(x[i]) <= shipment.loc[i, 'Weight']
    )

In [8]:
for j, row in holds.iterrows():
    ## Add constraints on each hold's weight capacity
    m.addConstr(
        gp.quicksum(
            x[k][j]
            for k in range(len(shipment))
        ) <= row['Weight']
    )
    ## Add constraints on each hold's space capacity
    m.addConstr(
        gp.quicksum(
            x[k][j] * shipment.loc[k, 'Volume']
            for k in range(len(shipment))
        ) <= row['Volume']
    )

In [9]:
## Add weight proportion constraints
for j, row in holds.iterrows():
    for i, row in holds.iterrows():
        if i > j:
            p1 = gp.quicksum(
                    x[k][j]
                    for k in range(len(shipment))
                ) / holds.loc[j, 'Weight']
            p2 = gp.quicksum(
                    x[k][i]
                    for k in range(len(shipment))
                ) / holds.loc[i, 'Weight']
            m.addConstr(p1 == p2)

In [10]:
## Add objective
m.setObjective(
    gp.quicksum(
        gp.quicksum(x[k]) * shipment.loc[k, 'Profit']
        for k in range(len(shipment))
    )
    ,GRB.MAXIMIZE
)

In [11]:
## Optimization
try: 
    m.optimize()
except gp.GurobiError as E:
    print("Optimize failed", E)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 25 rows, 12 columns and 72 nonzeros
Model fingerprint: 0x339a5256
Coefficient statistics:
  Matrix range     [6e-02, 6e+02]
  Objective range  [3e+02, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 9e+03]
Presolve removed 13 rows and 0 columns
Presolve time: 0.01s
Presolved: 12 rows, 12 columns, 52 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2750000e+04   4.329057e+03   0.000000e+00      0s
      14    1.2151579e+04   0.000000e+00   0.000000e+00      0s

Solved in 14 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.215157895e+04


In [12]:
for i in range(len(shipment)):
    for j in range(len(holds)):
        print('Distribute %f %s to %s'%(x[i][j].x, shipment.loc[i, 'Cargo'], holds.loc[j, 'Hold']))

Distribute 0.000000 C1 to Front
Distribute 0.000000 C1 to Center
Distribute 0.000000 C1 to Back
Distribute 10.000000 C2 to Front
Distribute 0.000000 C2 to Center
Distribute 5.000000 C2 to Back
Distribute 0.000000 C3 to Front
Distribute 12.947368 C3 to Center
Distribute 3.000000 C3 to Back
Distribute 0.000000 C4 to Front
Distribute 3.052632 C4 to Center
Distribute 0.000000 C4 to Back


In [13]:
total = sum([sum([x[i][j].x for j in range(len(holds))])*shipment.loc[i, 'Profit'] for i in range(len(shipment))])
#for i in range(len(shipment)):
print(total)  
#12151.57894736842 

12151.57894736842
